<a href="https://colab.research.google.com/github/ASTigran/BCS_SpeechEmotionRecognition/blob/main/MLPdraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#defining function to extract features from database

def f_ext(x,features,sr=10000):
  ft = np.array([])
  for f in features:
   
    if f=='chroma':
      chroma = librosa.ft.chroma_stft(x,sr=sr)
      ft = np.concatenate((ft,np.mean(chroma.T,axis=0)),axis=0)
    if f=='mfcc':
      mfccs = librosa.ft.mfcc(x,sr=sr )
      ft = np.concatenate((ft,np.mean(mfccs.T,axis=0)),axis=0)  
    if f=='mel':
      mel_spec = librosa.ft.melspectrogram(x)
      ft = np.concatenate((ft,np.mean(mel_spec.T,axis=0)),axis=0)
    
    
  return ft

 # dataset loading

def load_ds(type='emotion',features =['mel','mfcc','chroma']):                     
  dir = 'enter path here'
  X = []
  Y = []
  for j in os.listdir(dir):
    x,sr = librosa.load(dir+'/'+j)
    ft = f_ext(x,features)
    X.append(ft)
    if type =='emotion':                                
      y = int(j.split('-')[2][1])
      Y.append(y)
    else:
      y = np.array([((int(j.split('-')[6][1]))%2)])    
      Y.append(y)
  X = np.array(X)
  X = sklearn.preprocessing.scale(X)                  
  Y = np.array(Y).reshape(-1,1)
  encoder = OneHotEncoder()
  Y = encoder.fit_transform(Y).toarray()
  XY = np.hstack((X,Y))
  np.random.shuffle(XY)                              
  X = XY[:,:XY.shape[1]-Y.shape[1]]
  Y = XY[:,(XY.shape[1]-Y.shape[1]):]
  x_train = X[:math.ceil(train_size*len(X))]
  y_train = Y[:math.ceil(train_size*len(X))]
  x_test  = X[math.ceil(train_size*len(X)):]
  y_test = Y[math.ceil(train_size*len(X)):]

  return x_train,y_train,x_test,y_test

   #Plotting the losses and accuracy
def plotter(history):
  plt.figure()
  plt.plot(history.history['loss'],label='train loss')
  plt.plot(history.history['val loss'],label='test loss')
  plt.xlabel('iterations')
  plt.ylabel('losses')
  plt.legend()
  plt.figure()
  plt.plot(history.history['accuracy'],label='train accuracy')
  plt.plot(history.history['val accuracy'],label='test accuracy')
  plt.xlabel('iterations')
  plt.ylabel('accuracy')
  plt.legend()



x_train = np.load('.npy')
y_train = np.load('.npy')
x_test = np.load('.npy')
y_test = np.load('.npy')
x_test.shape

#building MLP model 
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(input_size, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=500,batch_size=1440)

plotter(history)

test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

predictions = model.predict(X_test)
np.argmax(predictions[0])

